In [1]:
# gradient boosting
import os
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRegressor
import networkx as nx

# https://catboost.ai/en/docs/concepts/tutorials
# https://youtu.be/usdEWSDisS0

In [2]:
NUMBER_NODES = 9

def load_data():
    train_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_train.csv'))
    val_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_val.csv'))
    test_df = pd.read_csv(os.path.join('..', 'datasets', f'dataset_{NUMBER_NODES}_test.csv'))

    featuresNumber = (NUMBER_NODES * NUMBER_NODES - NUMBER_NODES) // 2 
    def get_tuple_tensor_dataset(row):
        X = row[0 : featuresNumber].astype('int32')
        Y = row[featuresNumber + 1: ].astype('int32') # Inclui a banda otima na posicao 0
        return X, Y

    train_dataset = list(map(get_tuple_tensor_dataset, train_df.to_numpy()))
    val_dataset = list(map(get_tuple_tensor_dataset, val_df.to_numpy()))
    test_dataset = list(map(get_tuple_tensor_dataset, test_df.to_numpy()))

    X = []
    Y = []
    for x, y in train_dataset:
        X.append(x)
        Y.append(y)
    x_train = np.array(X)
    y_train = np.array(Y)

    X = []
    Y = []
    for x, y in test_dataset:
        X.append(x)
        Y.append(y)
    x_test = np.array(X)
    y_test = np.array(Y)

    X = []
    Y = []
    for x, y in val_dataset:
        X.append(x)
        Y.append(y)
    x_val = np.array(X)
    y_val = np.array(Y)

    x_train = np.concatenate((x_train, x_val))
    y_train = np.concatenate((y_train, y_val))


    return x_train, y_train, x_test, y_test

In [3]:
x_train, y_train, x_test, y_test = load_data()

In [4]:
cat_features = list(range(0, x_test.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]


In [5]:
model = CatBoostRegressor(objective='MultiRMSE', verbose=100)
model.fit(x_train, y_train, eval_set=(x_test, y_test), cat_features=cat_features, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 7.1096370	test: 7.1083527	best: 7.1083527 (0)	total: 352ms	remaining: 5m 51s
100:	learn: 6.8727177	test: 6.8750000	best: 6.8750000 (100)	total: 16.8s	remaining: 2m 29s
200:	learn: 6.7799744	test: 6.7873605	best: 6.7873605 (200)	total: 33.2s	remaining: 2m 12s
300:	learn: 6.7219209	test: 6.7334753	best: 6.7334753 (300)	total: 49.6s	remaining: 1m 55s
400:	learn: 6.6799716	test: 6.6946155	best: 6.6946155 (400)	total: 1m 5s	remaining: 1m 37s
500:	learn: 6.6455095	test: 6.6633350	best: 6.6633350 (500)	total: 1m 21s	remaining: 1m 21s
600:	learn: 6.6169507	test: 6.6376475	best: 6.6376475 (600)	total: 1m 37s	remaining: 1m 5s
700:	learn: 6.5927756	test: 6.6162984	best: 6.6162984 (700)	total: 1m 53s	remaining: 48.5s
800:	learn: 6.5717673	test: 6.5981994	best: 6.5981994 (800)	total: 2m 9s	remaining: 32.2s
900:	learn: 6.5536446	test: 6.5830679	best: 6.5830679 (900)	total: 2m 25s	remaining: 16s
999:	learn: 6.5369376	test: 6.5692003	best: 6.5692003 (999)	total: 2m 41s	remaining: 0us

bestTe

In [8]:
model.tree_count_

1000

In [6]:
def count_repeats(output):
    counts = np.unique(np.round(output))
    repeated = NUMBER_NODES - counts.shape[0]
    return repeated

def get_valid_pred(pred):
    valid = np.ones(NUMBER_NODES)
    labels = np.arange(0, NUMBER_NODES)
    for i in labels:
        min_value = np.amin(pred)
        min_idx, = np.where(pred == min_value)
        min_idx = min_idx[0]
        pred[min_idx] = 100
        valid[min_idx] = i
    return valid
    
def get_bandwidth(Graph, nodelist):
    Graph = nx.Graph(Graph)
    if not Graph.edges:
        return 0
    if nodelist.all() != None:
        L = nx.laplacian_matrix(Graph, nodelist=nodelist)
    else:
        L = nx.laplacian_matrix(Graph)
    x, y = np.nonzero(L)
    return (x-y).max()

def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj

In [7]:
pred = model.predict(x_test)

sumTest_original = 0
sumTest_pred = 0
sumTest_true = 0

count = 0
cases_with_repetition = 0

for i in range(len(pred)):

    output = pred[i]

    quantity_repeated = count_repeats(np.round(output))
    # print('Pred: ', output)
    # print('True: ', y_test[i])
    if quantity_repeated != 0:
        cases_with_repetition += 1
    output = get_valid_pred(output)
    # print('Pred valid: ', output)
    count += quantity_repeated

    # print("Bandwidth")
    graph = getGraph(x_test[i])
    original_band = get_bandwidth(graph, np.array(None))
    sumTest_original += original_band
    pred_band = get_bandwidth(graph, output)
    sumTest_pred += pred_band
    true_band = get_bandwidth(graph, y_test[i])
    sumTest_true += true_band
    # print("Bandwidth")
    # print(original_band)
    # print(pred_band)
    # print(true_band)
print('Quantidade de rótulos repetidos, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 6 - ', count)
print('Quantidade de saídas com repetição, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 1 - ', cases_with_repetition)
test_length = pred.shape[0]
print('Test length - ', test_length)
print("Bandwidth mean")
print(sumTest_original / test_length)
print("Pred bandwidth mean")
print(sumTest_pred / test_length)
print("True bandwidth mean")
print(sumTest_true / test_length)

Quantidade de rótulos repetidos, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 6 -  78379
Quantidade de saídas com repetição, exemplo [1, 1, 1, 1, 1, 1, 1] conta como 1 -  16480
Test length -  16480
Bandwidth mean
7.7975728155339805
Pred bandwidth mean
6.423240291262136
True bandwidth mean
4.092839805825243
